<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [ ]:
!pip install -q datasets pandas pillow tqdm huggingface_hub

In [41]:
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
from huggingface_hub import login
import getpass

In [42]:
# Authenticate Hugging Face
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(HF_TOKEN)

Enter your Hugging Face token: ··········


In [43]:
# Set download directory
BASE_DIR = Path.home() / "Downloads" / "deepfake_dataset"
BASE_DIR.mkdir(parents=True, exist_ok=True)
metadata = []

In [44]:
def download_data(dataset_name, category_name, file_ext, key):
    """Downloads and saves data (images, videos, audio)"""
    print(f"\n📥 Downloading {category_name} Data...")
    try:
        dataset = load_dataset(dataset_name, split="train", use_auth_token=True)
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {e}")
        return

    save_dir = BASE_DIR / category_name
    save_dir.mkdir(parents=True, exist_ok=True)

    for category, label in [("real", 0), ("fake", 1)]:
        (save_dir / category).mkdir(parents=True, exist_ok=True)
        samples = dataset.filter(lambda x: x["label"] == label).select(range(10))

        for i, sample in enumerate(tqdm(samples, desc=f"Saving {category} {category_name}")):
            file_path = save_dir / category / f"{category}_{i:03d}.{file_ext}"
            with open(file_path, "wb") as f:
                f.write(sample[key]["bytes"] if isinstance(sample[key], dict) else sample[key])

            metadata.append({
                "modality": category_name,
                "category": category,
                "filename": file_path.name,
                "file_path": str(file_path),
                "source_dataset": dataset_name
            })

def save_metadata():
    """Saves metadata as CSV"""
    pd.DataFrame(metadata).to_csv(BASE_DIR / "metadata.csv", index=False)
    print("✅ Metadata saved")

In [45]:
def main():
    download_data("deepfake-detection-challenge", "images", "jpg", "image")
    download_data("dawn-dai/deepfake-video", "videos", "mp4", "video")
    download_data("m4r4b0u/FakeAVCeleb", "audio", "wav", "audio")
    save_metadata()
    print(f"\n🎉 Dataset Collection Complete! Data stored in {BASE_DIR}")

if __name__ == "__main__":
    main()


📥 Downloading images Data...
❌ Error loading deepfake-detection-challenge: Dataset 'deepfake-detection-challenge' doesn't exist on the Hub or cannot be accessed.

📥 Downloading videos Data...
❌ Error loading dawn-dai/deepfake-video: Dataset 'dawn-dai/deepfake-video' doesn't exist on the Hub or cannot be accessed.

📥 Downloading audio Data...
❌ Error loading m4r4b0u/FakeAVCeleb: Dataset 'm4r4b0u/FakeAVCeleb' doesn't exist on the Hub or cannot be accessed.
✅ Metadata saved

🎉 Dataset Collection Complete! Data stored in /root/Downloads/deepfake_dataset
